Import libraries:

In [67]:
import numpy as np
import pandas as pd
from attention import AttentionLayer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

import warnings
import winsound
warnings.filterwarnings("ignore")

CDFs:

In [95]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'end'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['start']) and i!=target_word_index['end']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString = newString+reverse_source_word_index[i]+' '
    return newString

Parameters:

In [35]:
max_text_len = 30
max_summary_len = 8

Import data:

In [16]:
df = pd.read_csv("reviews/review.csv",nrows=100000)
df = df[['Text','Summary']]
df = df.rename(columns={'Text': 'text', 'Summary': 'summary'})

Text preprocessing

In [18]:
df.drop_duplicates(subset=['text'], inplace=True)
df.dropna(axis=0,inplace=True)
df.head(2)

,text,summary
0,I have bought several of the Vitality canned d...,Good Quality Dog Food
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised


Add start and end tokens:

In [19]:
df['summary'] = df['summary'].apply(lambda x : 'start '+ x + ' end')

Split into the train and test parts:

In [59]:
x_tr, x_val, y_tr, y_val = train_test_split(np.array(df['text']), np.array(df['summary']), 
                                            test_size=0.1, random_state=0, shuffle=True)

Tokenize texts:

In [60]:
x_tokenizer = Tokenizer(num_words=100) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

x_voc   =  x_tokenizer.num_words + 1

Tokenize summaries:

In [61]:
y_tokenizer = Tokenizer(num_words=100) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1

Create model:

In [78]:
from keras import backend as K 
K.clear_session()

latent_dim = 300
embedding_dim=100

# Encoder
encoder_inputs = Input(shape=(max_text_len,))
enc_emb =  Embedding(x_voc, embedding_dim, trainable=True)(encoder_inputs)

#encoder lstm
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 100)      10100       input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 30, 300), (N 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

Complile and introduce early stopping:

In [79]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

Fit transformer:

In [81]:
%%time
history = model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:], epochs=2,
                    callbacks=[es], batch_size=128, validation_data=([x_val,y_val[:,:-1]], 
                                                                    y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

winsound.Beep(2500, 1000)

Train on 79582 samples, validate on 8843 samples
Epoch 1/2
79582/79582 [==============================] - 1678s 21ms/sample - loss: 1.4033 - val_loss: 1.2871
Epoch 2/2
79582/79582 [==============================] - 1528s 19ms/sample - loss: 1.2526 - val_loss: 1.2247
Wall time: 53min 26s


Reverse tokenization:

In [87]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

Create prediction model:

In [93]:
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

Show summaries:

In [96]:
for i in range(0,100):
    print("Review:",seq2text(x_tr[i]))
    print("Original summary:",seq2summary(y_tr[i]))
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_text_len)))
    print("\n")

Review: is not only to use it only but is great the have just the of they up 
Original summary: 
Predicted summary:  end


Review: to my the food if you have a about in that is some had and some other by them in a it was a but the like i to on 
Original summary: excellent product not 
Predicted summary:  end


Review: coffee is but not make a of coffee even when you the of 
Original summary: 
Predicted summary:  coffee end


Review: when in this i of it but of them for the i was and they be i them so i them on and very with the product 
Original summary: one of my favorite 
Predicted summary:  end


Review: i'm am that i was to find these the were in and are what a find i be 
Original summary: of 
Predicted summary:  end


Review: i don't like it at all it very to me i in for a and to love if you are for the taste of coffee this is not it or 
Original summary: not to 
Predicted summary:  coffee end


Review: good i i like the when they are in this the are but all the a and well 
Origin

Predicted summary:  end


Review: you more the are on the of the no what you do but just be br br get a better or they use and the if it for it for 
Original summary: 
Predicted summary:  end


Review: other it the if you on a which i the of the is too for i would have this of had my just a is all great in that of 
Original summary: yummy 
Predicted summary:  end


Review: and i them the only with a is that you them in to them if you are not to them these are also to and a on them to as 
Original summary: and 
Predicted summary:  end


Review: these are and with a flavor they great with a of coffee for a or an 
Original summary: tasty 
Predicted summary:  end


Review: was dog food after food he to get a on it at because it and i was a than i i was very on this food it's a good product 
Original summary: dog food 
Predicted summary:  my dog loves it end


Review: there we get them at to my only for them is that them a but my my are they the there is not even a out there not them more 
